### 安裝Dash相關套件

In [ ]:
# pip install dash

In [ ]:
# pip install dash-html-components

In [ ]:
# pip install dash-core-components

In [ ]:
# pip install jupyter-dash

### 重要機構地圖資料準備

https://plotly.com/python/mapbox-layers/

In [1]:
# 準備捷運站相關資料
import pandas as pd
df = pd.read_csv("./done_data/subway_result.csv")
df_lng = df['經度'].astype('str').tolist()
df_lat = df['緯度'].astype('str').tolist()
df_name = df['出入口名稱'].astype('str').tolist()

In [2]:
# 準備公車站相關資料
df2 = pd.read_csv("./done_data/bus_result.csv")
# df2 = df2.head(100)
print(df2['經度'].shape, df2['緯度'].shape, df2['properties.bsm_chines'].shape)
df2_lng = df2['經度'].astype('str').tolist()
df2_lat = df2['緯度'].astype('str').tolist()
df2_name = df2['properties.bsm_chines'].astype('str').tolist()

(3187,) (3187,) (3187,)


In [3]:
# 準備醫療機構資料

def hospital(file_addr):
    result = pd.read_csv(file_addr, encoding = 'big5')
    return result.iloc[:,0:3]

hospital_df = pd.concat([hospital("./data/台北市診所清冊1090926(含經緯度).csv"), hospital("./data/台北市醫院清冊1090926(含經緯度).csv")], axis = 0)
hospital_df = pd.concat([hospital_df, hospital("./data/臺北市藥局清冊.csv")], axis = 0)
hospital_df.head()

,機構名稱,Longitude,Latitude
0,佳譽堂中醫診所,121.516999,25.030481
1,仲景中醫診所,121.527329,25.043628
2,華御中醫診所,121.531051,25.040480
3,日明耀中醫診所,121.513098,25.043609
4,華陽中醫診所,121.532434,25.043213


In [4]:
# 準備教育機構
education_1 = pd.read_csv("./data/taipeiPrivateSchool00.csv")
education_1 = education_1.iloc[:,1:4]
education_2 = pd.read_csv("./data/taipeiPublicKindergarten00.csv")
education_2 = education_2.iloc[:,0:3]
education_df = pd.concat([education_2,education_1], axis = 0)
education_df.head()

,Name,Longtitude,Latitude
0,臺北市松山區三民國民小學附設幼兒園,121.564280,25.063210
1,臺北市松山區民權國民小學附設幼兒園,121.562304,25.062387
2,臺北市松山區民族國民小學附設幼兒園,121.551798,25.059495
3,臺北市立松山幼兒園,121.551471,25.056748
4,臺北市松山區西松國民小學附設幼兒園,121.563488,25.052302


In [5]:
# 準備政府機構
gov_df = pd.read_csv("./data/taipeiGovernmentAgency01.csv")
gov_df = gov_df.iloc[:,0:3]
gov_df.head()

,Name,Longtitude,Latitude
0,臺北市政府,121.563997,25.037531
1,臺北市政府秘書處,121.563997,25.037531
2,臺北市市政大樓公共事務管理中心,121.563997,25.037531
3,臺北市政府民政局,121.564433,25.037542
4,臺北市孔廟管理委員會,121.516179,25.072762


In [6]:
# 準備2020的中古屋資料
house2020_df = pd.read_csv('./done_data/coord_house2020.csv', header = None)
house2020_df.columns = ['Longtitude','Latitude','Name']
house2020_df.head()

,Longtitude,Latitude,Name
0,121.565344,24.989453,臺北市文山區久康街91~120號
1,121.586374,24.989449,臺北市文山區政大二街171巷31~60號
2,121.5479611,25.0010014,臺北市文山區興隆路二段153巷5弄1~30號
3,121.5479611,25.0010014,臺北市文山區興隆路二段153巷5弄1~30號
4,121.5479611,25.0010014,臺北市文山區興隆路二段153巷5弄1~30號


### 各行政區資料準備

In [7]:
area_df = pd.read_csv("merge_df.csv")
area_df = area_df.set_index('Area')
area_df.columns

Index(['EightCount', 'ParkCount', 'FuneralCount', 'GasCount', 'CrimeCount',
       'PoliceCount', 'busCount', 'subwayCount', 'govCount', 'clinicCount',
       'hospitalCount', 'pharmacyCount', 'fireareaCount', 'firewayCount',
       '土地面積', '總人口數', '男性人數', '女性人數', '人口密度', '每戶人數', '每戶成年人數', '所得收入總計',
       '可支配所得', '消費支出', '儲蓄', '所得總額'],
      dtype='object')

Main app

In [8]:
import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import re

app = JupyterDash(__name__)

df = pd.DataFrame({
    "Category": ["Population", "Denm", "Bananas", "Apples", "Oranges", "Capital"],
    "Amount": [40000, 10000, 20000, 20000, 40000, 50000]
})

fig = px.bar(df, x="Category", y="Amount", barmode="group")


app.layout = html.Div([
    html.Div([
        html.H1(children='Housing Information in Taipei City', style = {
        }),
        html.P(children='Please click the symbol in the rigth side to select the facility you want', style = {
            'font-size':'16px'
        }),
        html.P(children='Taipei officially Taipei City, is the capital and a special municipality of Taiwan. Located in Northern Taiwan, Taipei City is an enclave of the municipality of New Taipei City that sits about 25 km southwest of the northern port city of Keelung. Most of the city rests on the Taipei Basin, an ancient lakebed. The basin is bounded by the relatively narrow valleys of the Keelung and Xindian rivers, which join to form the Tamsui River', style = {
            'font-size':'12px'
        }),
        dcc.Graph(
            id='gapminder',
            figure={
                'data': [
                    go.Scattermapbox(
                        lat=df_lat,
                        lon=df_lng,
                        mode='markers',
                        name="Metro Station",
                        marker=go.scattermapbox.Marker(
                size = 10, color='orange', symbol='rail-metro'
            ),
            text = df_name,
            ),
                    go.Scattermapbox(
                        lat = df2_lat,
                        lon = df2_lng,
                        mode = 'markers',
                        name="Bus Station",
                        marker = go.scattermapbox.Marker(
                size = 7, color='red',symbol='bus'
            ),
            text = df2_name,
            ),
            go.Scattermapbox(
                        lat=hospital_df['Latitude'],
                        lon=hospital_df['Longitude'],
                        mode='markers',
                        name="Health Facility",
                        marker=go.scattermapbox.Marker(
                size = 7, color='blue', symbol='hospital'
            ),
            text = hospital_df['機構名稱'],
            ),
            go.Scattermapbox(
                        lat=education_df['Latitude'],
                        lon=education_df['Longtitude'],
                        mode='markers',
                        name="Private Institutions and Schools",
                        marker=go.scattermapbox.Marker(
                size = 10, color='green', symbol='school'
            ),
            text = education_df['Name'],
            ),
            go.Scattermapbox(
                        lat=gov_df['Latitude'],
                        lon=gov_df['Longtitude'],
                        mode='markers',
                        name="Public Institutions and Schools",
                        marker=go.scattermapbox.Marker(
                size = 10, color='purple', symbol='police'
            ),
            text = gov_df['Name'],
            ),
            # 加入中古屋的資料
            go.Scattermapbox(
                        lat=house2020_df['Latitude'],
                        lon=house2020_df['Longtitude'],
                        mode='markers',
                        name="House Information",
                        marker=go.scattermapbox.Marker(
                size = 10, color='orange'
            ),
            text = house2020_df['Name'],
            )
                ],
            'layout': go.Layout(
                autosize=True,
                hovermode='closest',
                margin=dict(
                    l=20,
                    r=20,
                    b=20,
                    t=20
                ),
                mapbox=dict(
                    accesstoken='pk.eyJ1IjoiZXJpa3NvbjA2MTEiLCJhIjoiY2tpeTRib3RnMTd6dTJ5c2Joa3diZXVqcyJ9.2Qtsf3xtMppGs5lwXvDvyw',
                    bearing = 0,
                    center = dict(lat=25.03374536000858,lon=121.54328512547893),
                    pitch = 0,
                    zoom = 15
                    )) #layoutend
                }) # figure end
            ]), #div end
    html.Div([ #information div
        html.Div([
            html.H2("Districts Information"),
            html.Table([
                html.Tr([html.Td(['Population']), html.Td(id='output1')]),
                html.Tr([html.Td(['Population Density']), html.Td(id='output2')]),
                html.Tr([html.Td(['Number of Bus Stops']), html.Td(id='output3')]),
                html.Tr([html.Td(['Number of MRT Stations']), html.Td(id='output4')]),
                html.Tr([html.Td(['Number of Hospitals']), html.Td(id='output5')]), #hospitalCount
                html.Tr([html.Td(['Number of Gov Agencies']), html.Td(id='output6')]),# govCount
                html.Tr([html.Td(['Income per Capital']), html.Td(id='output7')])
                        ])
                    ],style={'width':'50%','display': 'inline-block'}),
        html.Div([
            html.H2("House Information"),
            html.P(id='hover-data'),
            ],style={'width':'50%', 'display': 'inline-block'})
    ]),

    dcc.Graph(
    id='example-graph',
    figure=fig
    )

], style={'marginLeft': 150, 'marginRight': 150, 'font-family':'微軟正黑體'})


@app.callback(
    dash.dependencies.Output('output1', 'children'),
    dash.dependencies.Output('output2', 'children'),
    dash.dependencies.Output('output3', 'children'),
    dash.dependencies.Output('output4', 'children'),
    dash.dependencies.Output('output5', 'children'),
    dash.dependencies.Output('output6', 'children'),
    dash.dependencies.Output('output7', 'children'),
    dash.dependencies.Input('gapminder', 'hoverData'))
def update_output(hoverData):
    if hoverData is None:
        return 'None','None','None','None','None','None','None'
    else:
        address = hoverData['points'][0]['text']
        try:
            result_match = re.search(r'(.{2}\u5340)', address)
            value = result_match[0]
            output1 = area_df['總人口數'][value]
            output2 = area_df['人口密度'][value]
            output3 = area_df['busCount'][value]
            output4 = area_df['subwayCount'][value]
            output5 = area_df['hospitalCount'][value]
            output6 = area_df['govCount'][value]
            output7 = area_df['所得總額'][value]
            return output1, output2, output3, output4, output5, output6, output7
        except:
            return 'None','None','None','None','None','None','None'

@app.callback(
    dash.dependencies.Output('hover-data', 'children'),
    dash.dependencies.Input('gapminder', 'hoverData'))
def display_hover_data(hover_data):
    if hover_data is None:
        return 'Please select a location'
    else:
        loaction = 'Loaction: {}'.format(hover_data['points'][0]['text'])
        return loaction

if __name__ == '__main__':
#     app.run_server(debug = True, mode = 'inline')
    app.run_server(debug = True)

Dash app running on http://127.0.0.1:8050/



### https://dash-gallery.plotly.host/dash-spatial-clustering/

In [ ]:
# 預計利用2020的資料，2020資料對應到資料表，房數、廳數、車位、建設材料、
# 下方條狀圖比較跟該區域的轉移坪數、單價、總價比較，統計平均數、中位數